In [ ]:
from rouge_score.rouge_scorer import _create_ngrams
from nltk.stem.porter import PorterStemmer
import spacy, six, json
from utils import tokenize
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

PorterStemmer = PorterStemmer()
nlp = spacy.load('en_core_web_sm')

In [ ]:

def plot_roc_curve(human_scores, gpt_scores):
    # Data
    A = human_scores
    B = gpt_scores
    # Combine scores and true labels
    scores = A + B
    labels = [0] * len(A) + [1] * len(B)
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(labels, scores)
    # Calculate AUC (Area Under Curve)
    roc_auc = auc(fpr, tpr)
    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: Open-gen w/ GPT3.5-Reddit w prompts' )
    plt.legend(loc="lower right")
    plt.show()
    # what is the TPR for FPR = 0.1?
    for idx, fpr_ in enumerate(fpr):
        if fpr_ > 0.01:
            print(f"TPR at 1% FPR: {tpr[idx]:.4f}")
            break

In [ ]:
def get_score_ngrams(target_ngrams, prediction_ngrams):
    intersection_ngrams_count = 0
    ngram_dict = {}
    for ngram in six.iterkeys(target_ngrams):
        intersection_ngrams_count += min(target_ngrams[ngram],
                                        prediction_ngrams[ngram])
        ngram_dict[ngram] = min(target_ngrams[ngram], prediction_ngrams[ngram])
    target_ngrams_count = sum(target_ngrams.values()) # prediction_ngrams
    return intersection_ngrams_count / max(target_ngrams_count, 1), ngram_dict


def get_ngram_info(article_tokens, summary_tokens, _ngram):
    article_ngram = _create_ngrams( article_tokens , _ngram)
    summary_ngram = _create_ngrams( summary_tokens , _ngram)
    ngram_score, ngram_dict = get_score_ngrams( article_ngram, summary_ngram) 
    return ngram_score, ngram_dict, sum( ngram_dict.values() )

In [ ]:
with open('results/regen_davinci003_20_0.5.jsonl', "r") as f:
    wop_regen_davinci003_20_05 = [json.loads(x) for x in f.read().strip().split("\n") ]
len(wop_regen_davinci003_20_05)

In [ ]:
wop_regen_davinci003_20_05[0].keys()

In [ ]:
len( wop_regen_davinci003_20_05[0]['gen_completion_regen']['choices'])#['choices'][1]['text']

In [ ]:
########################## with questions ##########################
ngram_overlap_count_wq = []
for idx, instance in enumerate( wop_regen_davinci003_20_05):
    human_07 = instance["gold_gen_truncate"] # gold_gen_truncate, human_gen_truncate
    gpt_07 = instance['gen_completion_truncate'] # gen_completion_truncate, machine_gen_truncate
    
    human_tokens = tokenize( human_07, stemmer=PorterStemmer)
    gpt_tokens = tokenize( gpt_07, stemmer=PorterStemmer)
    ########################################
    human_half = instance['gold_gen_regen']['choices']
    gpt_half = instance['gen_completion_regen']['choices']
    temp = []
    for i in range(20): # len(human_half)
        temp1 = {}
        temp2 = {}
        human_generate_tokens = tokenize(human_half[i]['text'], stemmer=PorterStemmer) # ['message']['content'] for chatgpt, ['text'] for gpt3
        gpt_generate_tokens = tokenize(gpt_half[i]['text'], stemmer=PorterStemmer )
        if len(human_generate_tokens) == 0 or len(gpt_generate_tokens) == 0:
            continue

        for _ngram in range(1, 25):
            ngram_score, ngram_dict, overlap_count = get_ngram_info(human_tokens, human_generate_tokens, _ngram)
            temp1['human_truncate_ngram_{}_score'.format(_ngram)] = ngram_score / len(human_generate_tokens)
            temp1['human_truncate_ngram_{}_count'.format(_ngram)] = overlap_count

            ngram_score, ngram_dict, overlap_count = get_ngram_info(gpt_tokens, gpt_generate_tokens, _ngram)
            temp2['gpt_truncate_ngram_{}_score'.format(_ngram)] = ngram_score / len(gpt_generate_tokens)
            temp2['gpt_truncate_ngram_{}_count'.format(_ngram)] = overlap_count
        temp.append({'human':temp1, 'machine':temp2})

    ngram_overlap_count_wq.append(temp)

In [ ]:
def N_gram_detector(ngram_n_ratio):
    score = 0
    non_zero = []
    
    for idx, key in enumerate(ngram_n_ratio):
        if idx in range(3) and 'score' in key or 'ratio' in key:
            score += 0. * ngram_n_ratio[ key ]
            continue
        if 'score' in key or 'ratio' in key:
            score +=  (idx+1) * np.log((idx+1))  * ngram_n_ratio[ key ]
            if ngram_n_ratio[ key ] != 0:
                non_zero.append( idx+1 ) 
    return score/ (sum( non_zero ) + 1e-8)

human_scores = []
gpt_scores = []

for instance in ngram_overlap_count_wq:
    human_score = []
    gpt_score = []

    for i in range(len(instance)):
        human_score.append( N_gram_detector(instance[i]['human'] ) )
        gpt_score.append( N_gram_detector(instance[i]['machine'] ) )

    human_scores.append( sum(human_score)  )
    gpt_scores.append( sum(gpt_score) )

In [ ]:
# plot and give different colors
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(human_scores, label='human')
plt.plot(gpt_scores, label='gpt')
plt.legend()
plt.show()

In [ ]:
plot_roc_curve(human_scores, gpt_scores)

In [ ]:
###### openai detector ######
with open( f'results/openai_detect_gpt4.jsonl', 'r') as f:
    openai_detect_gpt = [json.loads(x) for x in f.read().strip().split("\n")]
len(openai_detect_gpt)

In [ ]:
# plot and give different colors
import matplotlib.pyplot as plt
human_scores2 = [x['gold_gen_prob'] for x in openai_detect_gpt]
gpt_scores2 = [x['gen_completion_prob'] for x in openai_detect_gpt]

plt.figure(figsize=(10, 5))
plt.plot(human_scores2, label='human')
plt.plot(gpt_scores2, label='gpt')
plt.legend()
plt.show()

In [ ]:
plot_roc_curve(human_scores2, gpt_scores2)

In [ ]:
###### gptzero detector ######
with open( f'results/gptzero_gpt4.jsonl', 'r') as f:
    gptzero_detect_gpt = [json.loads(x) for x in f.read().strip().split("\n")]
len(gptzero_detect_gpt)

In [ ]:
# plot and give different colors
import matplotlib.pyplot as plt
human_scores2 = [x['gold_gen_prob'] for x in gptzero_detect_gpt]
gpt_scores2 = [x['gen_completion_prob'] for x in gptzero_detect_gpt]

plt.figure(figsize=(10, 5))
plt.plot(human_scores2, label='human')
plt.plot(gpt_scores2, label='gpt')
plt.legend()
plt.show()

In [ ]:
plot_roc_curve(human_scores2, gpt_scores2)